In [8]:
import pandas as pd 
import numpy as np
import sqlite3 as sql
import csv

In [13]:
df0=pd.read_json('china_sites_20240601.json')
df1=pd.read_json('站点列表.json')
df1.columns=df1.iloc[0]
df1=df1[1:]

In [15]:
df1

,监测点编码,监测点名称,城市,经度,纬度,对照点
1,1001A,万寿西宫,北京,116.3621,39.8784,N
2,1002A,定陵(对照点),北京,116.2202,40.2915,Y
3,1003A,东四,北京,116.4174,39.9289,N
4,1004A,天坛,北京,116.4072,39.8863,N
5,1005A,农展馆,北京,116.462,39.9365,N
...,...,...,...,...,...,...
2022,3734A,理想之城,东营,118.35,37.26,N
2023,3860A,皇山花园小区,临沂,118.3920,35.0300,N
2024,3861A,寒亭区实验一小,潍坊,119.2041,36.7699,N
2025,3866A,三高中,营口,122.2469,40.66302,N


In [16]:

df1.rename(columns={'监测点编码':'StationCode','监测点名称':'StationName','城市':'CITY','经度':'Longitude','纬度':'Latitude','对照点':'Contrast'},inplace=True)
df1.set_index('StationCode',inplace=True)


In [17]:
df0.drop(columns='date',inplace=True)
df0.set_index(['hour','type'],inplace=True)

In [18]:
df_new=df0.T
df_new.index.name="StationCode"
df_new.columns.names.clear()
df_new

hour         0                                                             \
type        AQI PM2.5 PM2.5_24h PM10 PM10_24h SO2 SO2_24h NO2 NO2_24h  O3   
StationCode                                                                 
1001A        36    14         6   36       15   3       3  31      14  86   
1002A        14     3         3   14        8   3       3  12      11  28   
1003A        31    11         4   31       13   2       2  28      12  84   
1004A        24     8         4   21       11   2       2  26      12  75   
1005A        28    10         4   28       12   1       2  28      15  82   
...          ..   ...       ...  ...      ...  ..     ...  ..     ...  ..   
3734A        34     9        19   34       49  10      12  39      21  47   
3860A        31    15        38   29       80   5      10  15      25  99   
3861A        29     7        28   19       62   7       9  13      19  91   
3866A        25     9        14   15       20   6       6  12      11  80   
3867A        59    33        19   67       51   6       8  20      19  94   

hour         ...  23                                                       \
type         ... SO2 SO2_24h NO2 NO2_24h   O3 O3_24h O3_8h O3_8h_24h   CO   
StationCode  ...                                                            
1001A        ...   1       2  27      27  131          165       161  0.4   
1002A        ...   1       2  16      16  109          133       127  0.3   
1003A        ...   4       3  25      30  125          112            0.4   
1004A        ...   3       3  22      22  106          156       142  0.3   
1005A        ...   2       2  22      29  116          135       133  0.3   
...          ...  ..     ...  ..     ...  ...    ...   ...       ...  ...   
3734A        ...  14      25  59      24   52          150       126  0.5   
3860A        ...   8       6  23      16   81          143       123  0.3   
3861A        ...   9       8  39      18   83          156       136  0.7   
3866A        ...   7       8  14      12  109          103       103  0.7   
3867A        ...   9       8  41      19   59          111        86  0.6   

hour                
type        CO_24h  
StationCode         
1001A          0.3  
1002A          0.2  
1003A          0.3  
1004A          0.3  
1005A          0.3  
...            ...  
3734A          0.4  
3860A          0.3  
3861A          0.5  
3866A          0.5  
3867A          0.5  

[2026 rows x 360 columns]

In [23]:
df_new.rename(columns={'PM2.5':'PM2_5','PM2.5_24h':'PM2_5_24h'},inplace=True)

In [24]:
# df1.index=pd.MultiIndex.from_product([df_new.index,['']],names=['hour','type'])
data_df=pd.concat([df1,df_new],axis=1)
data_df.dropna(inplace=True)
data_df

,StationName,CITY,Longitude,Latitude,Contrast,"(0, AQI)","(0, PM2_5)","(0, PM2_5_24h)","(0, PM10)","(0, PM10_24h)",...,"(23, SO2)","(23, SO2_24h)","(23, NO2)","(23, NO2_24h)","(23, O3)","(23, O3_24h)","(23, O3_8h)","(23, O3_8h_24h)","(23, CO)","(23, CO_24h)"
StationCode,,,,,,,,,,,,,,,,,,,,,
1001A,万寿西宫,北京,116.3621,39.8784,N,36,14,6,36,15,...,1,2,27,27,131,,165,161,0.4,0.3
1002A,定陵(对照点),北京,116.2202,40.2915,Y,14,3,3,14,8,...,1,2,16,16,109,,133,127,0.3,0.2
1003A,东四,北京,116.4174,39.9289,N,31,11,4,31,13,...,4,3,25,30,125,,112,,0.4,0.3
1004A,天坛,北京,116.4072,39.8863,N,24,8,4,21,11,...,3,3,22,22,106,,156,142,0.3,0.3
1005A,农展馆,北京,116.462,39.9365,N,28,10,4,28,12,...,2,2,22,29,116,,135,133,0.3,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3734A,理想之城,东营,118.35,37.26,N,34,9,19,34,49,...,14,25,59,24,52,,150,126,0.5,0.4
3860A,皇山花园小区,临沂,118.3920,35.0300,N,31,15,38,29,80,...,8,6,23,16,81,,143,123,0.3,0.3
3861A,寒亭区实验一小,潍坊,119.2041,36.7699,N,29,7,28,19,62,...,9,8,39,18,83,,156,136,0.7,0.5


In [25]:
data_df.to_csv('aqi_data.csv')

In [26]:
conn=sql.connect('AQI_data.db')
data_df.to_sql('aqi_data',conn,if_exists='replace',index=True)
conn.close()
print("DataBase has been created!")

DataBase has been created!
